## Семинар 11

Всем привет и добро пожаловать на наш семинар по сбору новостей. Сегодня мы ставим себе следующие цели:

1. Собрать все заголовки статей на определенную тему, отфильтровав по дате с сайта делового издания "Коммерсант"
2. Узнать про хитрости, которые могут помочь в сборе данных
3. Провести самостоятельную практику с похожими заданиями

Для этого мы воспользуемся уже знакомыми Вам библиотеками requests и BeautifulSoup, а также познакомимся с новой – Selenium.

*Материал подготовила Анастасия Максимовская, хитрости позаимствованы у Филиппа Ульянкина. Вопросы по материалам можно писать мне в телеграм @anastasiyamaxx*

<img src="https://avatars.mds.yandex.net/get-zen_doc/3892121/pub_5f7ab7fb8d3ae5589bb054aa_5f7ab85061e6d41ef5615d94/scale_1200" width=700>

## Забираем заголовки с Коммерсанта

In [1]:
import requests 
from bs4 import BeautifulSoup

Итак, начнем с простого – проверим, соберется ли у нас информация с главной страницы Коммерсанта или нужно искать специальные примочки.

In [2]:
url = 'https://www.kommersant.ru/'
response = requests.get(url)
response

<Response [200]>

`<Response [200]>` выглядит хорошо. Но имейте в виду – это не всегда значит, что мы получили нужную информацию. Например, когда я пишу этот семинар, главная страница выглядит так:


<img src='imgs/pic1.png' width=800>

Однако, если бы нам вылетел баннер (например, какое-нибудь предложение о скидке) или запрос в духе "уточните Ваше местоположение", или капча, то некоторый нужный текст с главной страницы в собранный html мог бы не попасть. Для этого можно либо глазами посмотреть на `response.content` или попробовать найти нужный элемент с помощью методов `.find()` (находит первый элемент, отвечающий условиям в скобочках) или `.find_all()` (находит все нужные элементы) из библиотеки `bs4`.

Сделаем деревце, по которому можно искать нужные элементы с помощью `bs4`:

In [3]:
tree = BeautifulSoup(response.content, 'html.parser')

Найдем главную новость в тексте. Для этого я перехожу на сайт Коммерсанта, навожу мышкой на заголовок "Роспотребнадзор сократил до суток максимальный срок выполнения исследования на коронавирус", щелкаю правой кнопкой мыши и нажимаю "Просмотреть код элемента". Вижу что-то такое:

<img src="imgs/pic2.png">

Попробуем найти этот элемент в нашем дереве!

In [4]:
tree.find_all('a', {'class': 'top_news_main__link link'})

[<a class="top_news_main__link link" href="/doc/5049497?from=top_main_1">
 <span class="vicon vicon--hot_news top_news_main__lightning">
 <svg class="vicon__body"><use xlink:href="#vicon-hot_news" xmlns:xlink="http://www.w3.org/1999/xlink"></use></svg>
 </span>
                         Минцифры хочет сделать обязательным использование платформы «Гостех» с 2024 года
                     </a>]

Достанем только текст:

In [9]:
tree.find('a', {'class': 'top_news_main__link link'}).text.strip()

'Минцифры хочет сделать обязательным использование платформы «Гостех» с 2024 года'

Однако, если Вы впервые заходите на сайт или откроете окно в режиме инкогните, то увидите, что при первом визите на сайт вылетает такой баннер:

<img src="imgs/pic3.png">

Также это можно заметить, полистав содержимое `tree`.

Конкретно в этом примере нам это не мешает вытащить заголовок. Однако, иногда такие всплывающие окна мешют собрать html с нужной информацией со страницы. Что же делать? Нам на помощь придет библиотека selenium – специальный инструмент для автоматизации действий браузера.

### Добавляем селениум :)

Библиотека `selenium` – набор инструментов для интерактивной работы в браузере средствами Python. Вообще Selenium ‒ это целый проект, в котором есть разные инструменты. Мы рассмотрим один из самых распространенных ‒ Selenium WebDriver, модуль, который позволяется Python встраиваться в браузер и работать в нем как пользователь: кликать на ссылки и кнопки, заполнять формы, выбирать опции в меню и прочее. 

In [9]:
# через восклицательный знак обращемся к командной строке (на маке называется terminal)
# pip – менеджер пакетов для питона, который позволяет нам поставить библиотеку
!pip install selenium
!pip install webdriver-manager

Для того, чтобы воспользоваться библиотекой, нужно загрузить вебдрайвер для Вашего браузера. Подробнее можно почитать [в пункте 1.5 документации про установку](https://selenium-python.readthedocs.io/installation.html). План действий такой: качате драйвер – прописываете путь в переменной PATH – используете.

Но мы воспользуемся лайфхаком, чтобы не мучиться долго с установкой. Это библиотека `webdriver-manager`, которая скачает вебдрайвер за Вас. Подробнее в [документации](https://pypi.org/project/webdriver-manager/) (там же можно посмотреть код для других браузеров).

In [10]:
from selenium.webdriver.common.keys import Keys

In [12]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [/Users/a18509896/.wdm/drivers/chromedriver/mac64/95.0.4638.17/chromedriver] found in cache


На Вашем компьютере откроется пустая страничка. Давайте перейдем на сайт Коммерсанта.

In [13]:
driver.get('https://www.kommersant.ru/')

Откройте эту страничку – теперь она не пустая :) 

Следующим шагом, нам надо понять, как кликнуть на баннер так, чтобы он закрылся. Для этого нужно определить пусть к кнопке. Как и раньше, наводим мышку и кликаем "Просмотреть код".

<img src="imgs/pic4.png">

Теперь нужно сделать 2 действия кодом:

1. Помочь драйверу найти элемент
2. Кликнуть на него

Есть несколько способов указать пусть к элементу, они описаны [здесь](https://selenium-python.readthedocs.io/locating-elements.html) (попросите Вашего семинариста вкратце прокомментировать каждый). 

Принципиальной разницы нам сейчас нет, предлагаю воспользоваться методом `driver.find_element_by_css_selector()`. Правой кнопокой мыши щелкните по коду нужной кнопки (принять или отклонить), выберите copy – copy selector.

<img src="imgs/pic5.png" width=500>

Сохраним селектор в переменную, найдем нужный элемент и кликнем. Иногда работает не с первого раза.

In [14]:
selector = "body > div.subscription-popup-v2.subscription-popup-v2_push-popup > div.subscription-popup-v2__inner-container > div.subscription-popup-v2__controls > div.subscription-popup-v2__reject"

In [15]:
ss = driver.find_element_by_css_selector(selector)

In [16]:
ss

<selenium.webdriver.remote.webelement.WebElement (session="bf36017ee64a645376ff65a9e61e32e0", element="4f40e84d-fe1c-4467-8231-7b80dba801f5")>

In [17]:
ss.click()

Обновим страничку на всякий случай:

In [18]:
driver.refresh()

Давайте найдем главный заголовок еще одним способом. Сначала найдем элемент, помня имя класса (см. скрины выше), потом достанем его html код.

In [19]:
main_news = driver.find_element_by_class_name("top_news_main__name")
main_news

<selenium.webdriver.remote.webelement.WebElement (session="bf36017ee64a645376ff65a9e61e32e0", element="71c5f8d6-fb84-4ea1-837e-c3cc5945bcaa")>

In [20]:
main_news.get_attribute('innerHTML')

'\n                    <a href="/doc/5049497?from=top_main_1" class="top_news_main__link link">\n                        <span class="vicon vicon--hot_news top_news_main__lightning">\n                            <svg class="vicon__body"><use xmlns:xlink="http://www.w3.org/1999/xlink" xlink:href="#vicon-hot_news"></use></svg>\n                        </span>\n                        Минцифры хочет сделать обязательным использование платформы «Гостех» с 2024 года\n                    </a>\n                '

In [21]:
small_tree = BeautifulSoup(main_news.get_attribute('innerHTML'), 'html.parser')

In [22]:
small_tree


<a class="top_news_main__link link" href="/doc/5049497?from=top_main_1">
<span class="vicon vicon--hot_news top_news_main__lightning">
<svg class="vicon__body"><use xlink:href="#vicon-hot_news" xmlns:xlink="http://www.w3.org/1999/xlink"></use></svg>
</span>
                        Минцифры хочет сделать обязательным использование платформы «Гостех» с 2024 года
                    </a>

In [24]:
small_tree.text.strip()

'Минцифры хочет сделать обязательным использование платформы «Гостех» с 2024 года'

Ура, получили заголовок главной новости. Если он отличается от того, что на скрине, то нестрашно – новостные сайты быстро меняют статьи на главных страницах. Остальное можно достать по аналогии :)

Перейдем к более интересному – соберем все новости на определенную тему и срок.

Предлагаю попробовать собрать все новости, содержащие фразу "центральный банк" за период с 24 августа 2021 по текущий день. То есть, переводя это на программистский, нам нужно проделать следующие действия:

1. Найти окно поиска, кликнуть
2. Ввести в него ключевую фразу, нажать кнопку поиска
3. Нажать кнопку расширенный поиск
4. Найти кнопку, где изменяем дату начала поиска, выставить нужную нам
5. Собрать информацию

Давайте начнем :) В прошлый раз мы воспользовались поиском с помощью селектора `.find_element_by_css_selector()`. Теперь добавим немного разнообразия и сделаем поиском через XPath. Получить ее можно по старой схеме: наводим мышь на окно поиска – кликаем посмотреть код – правой кнопкой кликаем по мыши на выделенном коде – выбираем copy – copy xpath.   

По шагам:

1. наводим мышь на окно поиска – кликаем посмотреть код
<img src="imgs/pic6.png" width=800 alt="aa"> 
2. правой кнопкой мыши кликаем на выделенном коде – выбираем copy – copy xpath   
<img src="imgs/pic7.png" width=500> 

In [24]:
"Гарик "Бульдог" Харламов"

SyntaxError: invalid syntax (<ipython-input-24-1846875b534c>, line 1)

In [25]:
"Гарик \"Бульдог\" Харламов"

'Гарик "Бульдог" Харламов'

In [26]:
'Гарик "Бульдог" Харламов'

'Гарик "Бульдог" Харламов'

In [25]:
# найденный по инструкции выше xpath к лупе
xpath_query = '//*[@id="js-navsearch-submit"]'
# находим окно поиска
search = driver.find_element_by_xpath(xpath_query)
# кликаем на него
search.click()

In [26]:
# найденный по инструкции выше xpath к окну поиска
xpath_query = '//*[@id="js-navsearch-query"]'
# находим окно поиска
search = driver.find_element_by_xpath(xpath_query)
# кликаем на него
search.click()

In [27]:
search_term = "центральный банк"
# печатаем фразу для поиска в окне для поиска
search.send_keys(search_term)

In [28]:
# нажимаем кнопку enter
search.send_keys(Keys.RETURN)

Если Вы посмотрите, что происходит в окне браузера, которым управляет селениум, то увидите, что окно поменялось и мы теперь в разделе поиска :) 

<img src="imgs/pic8.png" width=500>

Нажимаем на кнопку расширенный поиск и выбираем дату.

Дальше мы все уже знаем. Откройте в соседней с ноутбуком вкладке сайт коммерсанта и доставайте оттуда нужные селекторы / xpath (неважно).

In [29]:
# находим селектор для кнопки расширенный поиск и нажимаем ее
selector2 = "body > main > div > div > section > div.grid-col.grid-col-s3 > form > div.ui-field_pack > label"


In [30]:
ext_search = driver.find_element_by_css_selector(selector2)

In [31]:
ext_search.click()

In [32]:
# находим селектор для поля ввода даты
selector3 = "body > main > div > div > section > div.grid-col.grid-col-s3 > form > div.ui-collapse.js-toggle-collapse.js-toggle-item.ui-collapse--show > section.simple_page_section.simple_page_section--form.js-search-settings > div.grid.ui-collapse.js-toggle-collapse.js-toggle-item.ui-collapse--show > div:nth-child(1) > div > input"


In [33]:
date = driver.find_element_by_css_selector(selector3)

Обратите внимание на картинку ниже – дата начала поиска по дефолту вбита в окошко, надо ее удалить.

<img src="imgs/pic9.png" width=500>

In [34]:
# удаляем введеный по дефолту текст в ячейке
date.send_keys(Keys.SHIFT, Keys.END, Keys.BACK_SPACE)

In [35]:
# вводим нужную дату и надижимаем enter
date_start = "24.08.2021"
date.send_keys(date_start)

In [36]:
date.send_keys(Keys.RETURN)

In [37]:
driver.close()

Ура, получили нужную выдачу! Попробуем перейти на следующую страничку.

In [45]:
# путь к кнопке следующая страница
xpath3 = "/html/body/main/div/div/section/div[1]/div[3]/a"

In [46]:
second_page = driver.find_element_by_xpath(xpath3)

In [47]:
second_page.click()

Посмотрим на адрес нашей странички:

In [48]:
driver.current_url

'https://www.kommersant.ru/search/results?search_query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&dateStart=2021-08-24&dateEnd=2021-10-16&page=2'

In [50]:
# driver.page_source

Обратите внимание на параметр `page=2`. Если мы будем менять номера, то будем перемещаться по всем страницам с заголовками, удовлетворяющим нашим условиям. Осталось написать функцию, которая будет доставать нужную информацию с одной странички, и запустить ее циклом для всех.

Начнем с того, как задать url. Обратите внимание на обратный слэш, это так называемый line continuation character. Он означает, что код продолжится на следующей строке. Также обратите внимание на букву f перед продолжением url-адреса на 3 строчке – она позваоляет мне подставить значение переменной `{page_num}` в середину строки.

In [51]:
page_num = 1
url = 'https://www.kommersant.ru/search/results?search_query='\
    '%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1'\
    '%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&'\
    f'dateStart=2021-08-24&dateEnd=2021-10-15&page={page_num}'

In [52]:
url

'https://www.kommersant.ru/search/results?search_query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&dateStart=2021-08-24&dateEnd=2021-10-15&page=1'

Как обычно забираем HTML-разметку и делаем деревце бьютифул супом.

In [53]:
response2 = requests.get(url)
response2

<Response [200]>

In [54]:
tree_search = BeautifulSoup(response2.content, 'html.parser')

Уже знакомый по лекциям механизм поиска элемента по html разметке.

In [55]:
# находим заголовки
headers = tree_search.find_all('h2', {'class': 'uho__name rubric_lenta__item_name'})

In [57]:
len(headers)

10

In [58]:
headers[0]

<h2 class="uho__name rubric_lenta__item_name">
<a class="uho__link uho__link--overlay" href="/doc/5007292?query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9%20%D0%B1%D0%B0%D0%BD%D0%BA" target="_blank">«Одна из основных проблем — монетарная политика <mark>центральных</mark> <mark>банков</mark>»</a>
</h2>

Если достать из тега текст, то можно заметить, что есть пробелы / переходы на новые строки в начале и конце. Метод `.strip()` избавится от них.

In [59]:
headers[0].text

'\n«Одна из основных проблем — монетарная политика центральных банков»\n'

In [60]:
headers[0].text.strip()

'«Одна из основных проблем — монетарная политика центральных банков»'

In [61]:
# находим подзаголовки
subheaders = tree_search.find_all('h3', \
                {'class': 'uho__subtitle rubric_lenta__item_subtitle'})

In [62]:
len(subheaders)

6

Подзаголовки есть не у всех новостей!

In [63]:
subheaders[0]

<h3 class="uho__subtitle rubric_lenta__item_subtitle">
<a class="uho__link" href="/doc/5007292?query=%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9%20%D0%B1%D0%B0%D0%BD%D0%BA" target="_blank">Олег Богданов — о причинах роста стоимости газа в Европе</a>
</h3>

In [64]:
subheaders[0].text

'\nОлег Богданов — о причинах роста стоимости газа в Европе\n'

In [65]:
subheaders[0].text.strip()

'Олег Богданов — о причинах роста стоимости газа в Европе'

In [66]:
# напишем функцию, которая будет выдавать список из словарей 
# в каждом словаре заголовок и описание
def get_page_info(page_num):
    url = 'https://www.kommersant.ru/search/results?search_query='\
        '%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1'\
        '%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&'\
        f'dateStart=2021-08-24&dateEnd=2021-10-15&page={page_num}'
    
    response = requests.get(url)
    tree_search = BeautifulSoup(response.content, 'html.parser')
    headers = tree_search.find_all('h2', \
                                   {'class': 'uho__name rubric_lenta__item_name'})
    subheaders = tree_search.find_all('h3', \
                            {'class': 'uho__subtitle rubric_lenta__item_subtitle'})
    result = []
    for i in range(len(headers)):
        header = headers[i]
        try:
            subheader = subheaders[i]
            d = {'article_header': header.text.strip(),
                 'article_subheader': subheader.text.strip()}
        except:
            d = {'article_header': header.text.strip(),
                 'article_subheader': ''}
        result.append(d)
    return result

In [67]:
all_data = []
for n in range(1, 14):
    all_data.extend(get_page_info(n))

In [68]:
len(all_data)

130

Пока не очень знакомая библиотека сделает табличку из списка и позволит сохранить ее в файл.

In [69]:
import pandas as pd

In [70]:
df = pd.DataFrame(all_data)

In [71]:
df.head()

,article_header,article_subheader
0,«Одна из основных проблем — монетарная политик...,Олег Богданов — о причинах роста стоимости газ...
1,«Платина» решила взять ЦБ силой,Ксения Дементьева о методах борьбы банков за в...
2,Банк «Открытие» планирует размещение трехлетни...,Объем нового выпуска серии БО-П09 составит не ...
3,Пункты плана подготовки Банком России обзора ДКП,Как к нему подготовиться и как пережить
4,Банк Норвегии первым из крупных западных ЦБ по...,Чего ожидать инвесторам и вкладчикам от повыше...


In [235]:
# сохранить в csv формат
# index=False сделает так, чтобы колонка с индексами не вогла в итоговый файл
df.to_csv('all_data.csv', index=False)

In [72]:
# сохранить в xlsx формат
df.to_excel('all_data.xlsx', index=False)

In [73]:
# не забываем закрыть браузер драйвером после завершения работы :)
driver.close()

## Практика

Основная цель практики – убедиться, что Вам понятен код с семинара. Попробуйте собрать информацию по аналогии с сайта Коммерсанта. Фразу для поиска оставляем на Ваш выбор, срок – за последний месяц.